In [ ]:
# default_exp __init__

In [ ]:
# hide
import os
notebooks_dir = os.getcwd()
project_dir = os.path.dirname(notebooks_dir)

import sys
sys.path.append(project_dir)

# Trading Specifications Fetcher

In [ ]:
from ccstabilizer import MXCAPI

In [ ]:
# export
import os
import time
import yaml


class Fetcher(object):

    LIQUIDITY_YAML = './crypto-liquidity.yaml'
    TRADING_SPEC_YAML = './crypto-trading-spec.yaml'

    def __init__(self, exchange):
        self.mxc = MXCAPI(os.environ['MXC_ACCESS_KEY'], os.environ['MXC_SECRET_KEY'])
        self.exchange = exchange

    @staticmethod
    def write(yaml_data, yaml_file_path):
        with open(yaml_file_path, 'w') as f:
            yaml.dump(yaml_data, f, sort_keys=False)

    @staticmethod
    def read(yaml_file_path):
        if not os.path.exists(yaml_file_path):
            return {}
        with open(yaml_file_path) as f:
            return yaml.load(f, Loader=yaml.FullLoader)

    @staticmethod
    def extract_symbols(liquidity):
        symbol_and_liquid = liquidity.items()
        symbols = {
            symbol for symbol, _ in symbol_and_liquid
        }
        liquid_symbols = {
            symbol for symbol, liquid in symbol_and_liquid if liquid == True
        }
        return symbols, liquid_symbols

    def fetch_symbols(self):
        liquidity = {
            trading_spec['symbol']: trading_spec.get('limited', False) for trading_spec in self.mxc.get_symbols()
        }
        type(self).write(liquidity, type(self).LIQUIDITY_YAML)
        return type(self).extract_symbols(liquidity)

    def read_symbols(self):
        liquidity = type(self).read(type(self).LIQUIDITY_YAML)
        return type(self).extract_symbols(liquidity)

    def get_trading_spec(self, crypto_symbol, fiat_symbol):
        trading_spec = self.exchange.get_trading_specification(crypto_symbol, fiat_symbol)
        trading_specs = type(self).read(type(self).TRADING_SPEC_YAML)
        trading_specs.update({
            f'{crypto_symbol}-{fiat_symbol}': trading_spec
        })
        type(self).write(trading_specs, type(self).TRADING_SPEC_YAML)
        return trading_spec

In [ ]:
from ccstabilizer import MXC
from ccstabilizer import secrets
exchange = MXC()

In [ ]:
Fetcher(exchange).get_trading_spec('AR', 'USDT')

{'min_trade_unit': Decimal('0.01'),
 'min_trade_fiat_money_limit': Decimal('5'),
 'fee_rate': Decimal('0.002'),
 'liquid': True}

In [ ]:
from ccstabilizer import Binance
from ccstabilizer import secrets
exchange = Binance()

In [ ]:
Fetcher(exchange).get_trading_spec('CELO', 'USDT')

{'min_trade_unit': Decimal('0.01000000'),
 'min_trade_fiat_money_limit': Decimal('10.00000000'),
 'fee_rate': Decimal('0.001'),
 'liquid': True}